In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

### Configure the model training policy

In [2]:
model_development.configure_training_policy()

### Load and preprocess data

In [3]:
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv')
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

### Prepare data for model training

In [4]:
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


### Train classification model (first phase)

In [5]:
optimizer = model_development.create_optimizer('nadam')
resnet152v2 = \
    model_development.resnet152v2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = resnet152v2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'ResNet152v2', version = '1.00.00_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 1.0403 - accuracy: 0.6670 - precision: 0.6819 - recall: 0.5273
Epoch 1: val_accuracy improved from -inf to 0.68955, saving model to ../models/weights\ResNet152v2_1.00.00_fs.hdf5
119/119 [==============================] - 45s 298ms/step - loss: 1.0403 - accuracy: 0.6670 - precision: 0.6819 - recall: 0.5273 - val_loss: 0.8387 - val_accuracy: 0.6896 - val_precision: 0.7059 - val_recall: 0.4932
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5461 - accuracy: 0.7119 - precision: 0.7712 - recall: 0.5376
Epoch 2: val_accuracy did not improve from 0.68955
119/119 [==============================] - 29s 241ms/step - loss: 0.5461 - accuracy: 0.7119 - precision: 0.7712 - recall: 0.5376 - val_loss: 1.4749 - val_accuracy: 0.4478 - val_precision: 0.4400 - val_recall: 0.9795
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5161 - accuracy: 0.7393 - precision: 0.7637 - recall: 0.6333


### Train classification model (second phase)

In [6]:
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/ResNet152v2_1.00.00_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'ResNet152v2', version = '1.00.00_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 3.9765 - accuracy: 0.5963 - precision: 0.5546 - recall: 0.6538
Epoch 1: val_accuracy improved from -inf to 0.67761, saving model to ../models/weights\ResNet152v2_1.00.00_ss.hdf5
119/119 [==============================] - 91s 603ms/step - loss: 3.9765 - accuracy: 0.5963 - precision: 0.5546 - recall: 0.6538 - val_loss: 0.6809 - val_accuracy: 0.6776 - val_precision: 0.6145 - val_recall: 0.6986
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5485 - accuracy: 0.7145 - precision: 0.7851 - recall: 0.5285
Epoch 2: val_accuracy did not improve from 0.67761
119/119 [==============================] - 67s 562ms/step - loss: 0.5485 - accuracy: 0.7145 - precision: 0.7851 - recall: 0.5285 - val_loss: 1.1250 - val_accuracy: 0.5343 - val_precision: 0.4823 - val_recall: 0.9315
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5196 - accuracy: 0.7330 - precision: 0.8090 - recall: 0.5547
